# What happens:
File is created and email is prepared

## Import 

In [1]:
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from sqlalchemy import create_engine
import win32com.client as win32
from openpyxl import Workbook, load_workbook

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

## Load template

In [4]:
im_folder = "Import/"
ex_folder = "Export/"
file_path = "NPE_Capital_Template.xlsx"
file_name = "EAD_NPE_Backstop_" + rd + ".xlsx"
npe_template = load_workbook(im_folder + file_path)
npe_template_sql = npe_template['SQL']
npe_sql_tcdr = npe_template_sql['A2'].value
npe_sql_cet1 = npe_template_sql['A6'].value

### Save selection
Also see how much time it took

In [5]:
%%time
d1 = pd.read_sql_query(npe_sql_tcdr, connection_RAY)
d2 = pd.read_sql_query(npe_sql_cet1, connection_RAY)

CPU times: total: 3.52 s
Wall time: 1min 33s


In [6]:
# Generating workbook
ExcelWorkbook = load_workbook(filename = im_folder + file_path)
 
# Generating the writer engine
writer = pd.ExcelWriter(ex_folder + file_name, engine = 'openpyxl')
# Assigning the workbook to the writer engine
writer.book = ExcelWorkbook
d1.to_excel(writer, sheet_name = "TCDR", index=False)
d2.to_excel(writer, sheet_name = "CET1-Deductions", index=False)

ExcelWorkbook.save(ex_folder + file_name)
ExcelWorkbook.close()

C:\Users\WZHARBC\AppData\Local\Temp\1\ipykernel_16456\1528375210.py:7: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = ExcelWorkbook


### Send the email 

In [ ]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'stefan.wagner@rbinternational.com;mathias.mesicek@rbinternational.com;lediana.kalaja@rbinternational.com;olga.muster@rbinternational.com; tea.kranner@rbinternational.com'
mail.Cc = 'nertila.gjakova@rbinternational.com'
mail.Subject = 'NPE-Capital Deduction for ' + rd[4:6] + "/" + rd[2:4]
mail.Body = '''Dear colleagues, 

Attached you can find  the ''' + rd[4:6] + '''/''' + rd[2:4] + ''' results.

Kr,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + ex_folder + file_name
mail.Attachments.Add(attachment)

mail.Display()